# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
from datetime import timedelta
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Format today and yesterday into strings for filename creation
format = "%m-%d-%Y"
now = datetime.now()
today = now.strftime(format)
yesterday = (now - timedelta(days=1)).strftime(format)

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    jh_live_date = today
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    jh_live_date = yesterday
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
print(jh_live_date)

09-16-2022


In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2022-09-17 04:20:19,33.93911,67.709953,196992,7794,NaN,NaN,Afghanistan,506.037491,3.956506
1,NaN,NaN,NaN,Albania,2022-09-17 04:20:19,41.15330,20.168300,331459,3586,NaN,NaN,Albania,11517.791368,1.081883
2,NaN,NaN,NaN,Algeria,2022-09-17 04:20:19,28.03390,1.659600,270599,6879,NaN,NaN,Algeria,617.086805,2.542138
3,NaN,NaN,NaN,Andorra,2022-09-17 04:20:19,42.50630,1.521800,46147,155,NaN,NaN,Andorra,59725.619621,0.335883
4,NaN,NaN,NaN,Angola,2022-09-17 04:20:19,-11.20270,17.873900,103131,1917,NaN,NaN,Angola,313.789810,1.858801


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/7/22,9/8/22,9/9/22,9/10/22,9/11/22,9/12/22,9/13/22,9/14/22,9/15/22,9/16/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,195012,195298,195471,195631,195925,196182,196404,196751,196870,196992
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,330516,330687,330842,330948,331036,331053,331191,331295,331384,331459
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,270489,270507,270522,270532,270539,270551,270551,270570,270584,270599


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/7/22,9/8/22,9/9/22,9/10/22,9/11/22,9/12/22,9/13/22,9/14/22,9/15/22,9/16/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7785,7786,7786,7787,7787,7789,7791,7792,7792,7794
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3584,3584,3584,3584,3584,3585,3586,3586,3586,3586
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6879,6879,6879,6879,6879,6879,6879,6879,6879,6879


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/7/22,9/8/22,9/9/22,9/10/22,9/11/22,9/12/22,9/13/22,9/14/22,9/15/22,9/16/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/7/22,9/8/22,9/9/22,9/10/22,9/11/22,9/12/22,9/13/22,9/14/22,9/15/22,9/16/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,18125,18233,18233,18233,18233,18233,18233,18233,18303,18303
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,64672,65088,65088,65088,65088,65088,65088,65088,65353,65353
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,6757,6826,6826,6826,6826,6826,6826,6826,6873,6873


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/7/22,9/8/22,9/9/22,9/10/22,9/11/22,9/12/22,9/13/22,9/14/22,9/15/22,9/16/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,223,226,226,226,226,226,226,226,227,227
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,699,702,702,702,702,702,702,702,705,705
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,101,101,101,101,101,101,101,101,101,101


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
678,1001.0,Autauga,Alabama,US,2022-09-17 04:20:19,32.539527,-86.644082,18303,227,NaN,NaN,"Autauga, Alabama, US",32760.564893,1.240234
715,1075.0,Lamar,Alabama,US,2022-09-17 04:20:19,33.779950,-88.096680,4350,69,NaN,NaN,"Lamar, Alabama, US",31510.322347,1.586207
716,1077.0,Lauderdale,Alabama,US,2022-09-17 04:20:19,34.901719,-87.656247,28080,405,NaN,NaN,"Lauderdale, Alabama, US",30281.788869,1.442308


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,95645794,1053389,0.0
India,44528524,528302,0.0
France,35076991,155712,0.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,95645794,1053389,0.0,94592405.0,2022-09-17 04:20:19,37.934269,-91.444083
India,44528524,528302,0.0,44000222.0,2022-09-17 04:20:19,23.088275,81.806127
France,35076991,155712,0.0,34921279.0,2022-09-17 04:20:19,6.430808,-34.730285
Brazil,34568833,685203,0.0,33883630.0,2022-09-17 04:20:19,-12.669522,-48.480493
Germany,32680355,148933,0.0,32531422.0,2022-09-17 04:20:19,51.571844,10.277881


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,11199214,95550,0.0
Texas,7949154,90639,0.0
Florida,7109769,80782,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,11199214,95550,0.0,11103664.0,2022-09-17 04:20:19,37.843962,-120.728594
Texas,7949154,90639,0.0,7858515.0,2022-09-17 04:20:19,31.660643,-98.653069
Florida,7109769,80782,0.0,7028987.0,2022-09-17 04:20:19,28.940755,-82.700744
New York,6053818,71502,0.0,5982316.0,2022-09-17 04:20:19,42.544151,-75.474183
Illinois,3733902,39536,0.0,3694366.0,2022-09-17 04:20:19,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,3437648,33447,0.0
Florida,Miami-Dade,1450945,11648,0.0
Arizona,Maricopa,1421340,17865,0.0
Illinois,Cook,1394936,14766,0.0
Texas,Harris,1209407,11278,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,3437648,33447,0.0,3404201.0,2022-09-17 04:20:19,34.308284,-118.228241,6037.0
Florida,Miami-Dade,1450945,11648,0.0,1439297.0,2022-09-17 04:20:19,25.611236,-80.551706,12086.0
Arizona,Maricopa,1421340,17865,0.0,1403475.0,2022-09-17 04:20:19,33.348359,-112.491815,4013.0
Illinois,Cook,1394936,14766,0.0,1380170.0,2022-09-17 04:20:19,41.841448,-87.816588,17031.0
Texas,Harris,1209407,11278,0.0,1198129.0,2022-09-17 04:20:19,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-09-14,196751,331295,270570,46147,103131,11,9008,9697763,439302,10122875,...,982846,243993,11930,543930,11448034,702591,535,11932,333313,256939
2022-09-15,196870,331384,270584,46147,103131,11,9008,9697763,439302,10150636,...,982846,244006,11930,544057,11450999,702768,535,11932,333337,256939
2022-09-16,196992,331459,270599,46147,103131,11,9008,9697763,439302,10157316,...,982846,244023,11937,544090,11454079,702768,535,11932,333363,256939


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-09-14,7792,3586,6879,155,1917,0,145,129830,8669,14458,...,7462,1637,14,5809,43132,5706,0,2155,4017,5596
2022-09-15,7792,3586,6879,155,1917,0,145,129830,8669,14624,...,7462,1637,14,5809,43137,5707,0,2155,4017,5596
2022-09-16,7794,3586,6879,155,1917,0,145,129830,8669,14749,...,7462,1637,14,5809,43137,5707,0,2155,4017,5596


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-09-14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-09-15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-09-16,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/7/22,9/8/22,9/9/22,9/10/22,9/11/22,9/12/22,9/13/22,9/14/22,9/15/22,9/16/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,18125,18233,18233,18233,18233,18233,18233,18233,18303,18303
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,4303,4335,4335,4335,4335,4335,4335,4335,4350,4350
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,27767,27951,27951,27951,27951,27951,27951,27951,28080,28080
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,8700,8762,8762,8762,8762,8762,8762,8762,8815,8815
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,43516,43774,43774,43774,43774,43774,43774,43774,43969,43969


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-09-14,1504180,296805,8217,2264159,943944,11140012,1642492,885767,305284,49,...,2321550,7937740,1032129,141282,23018,2066675,1796023,593806,1848929,175746
2022-09-15,1512134,296805,8217,2264159,944789,11193606,1644829,886348,305517,49,...,2323074,7938813,1034616,141282,23031,2068350,1796023,594355,1850553,175746
2022-09-16,1512134,296805,8217,2264159,945449,11199214,1645456,887501,305772,49,...,2324658,7949154,1034616,141282,23049,2070182,1800799,595044,1851886,175746


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2022-09-14       18233   65088    6826  7450  16673    2783   5975   36319   
2022-09-15       18303   65353    6873  7481  16793    2788   6021   36775   
2022-09-16       18303   65353    6873  7481  16793    2788   6021   36775   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-09-14         9925     5931  ...    7393   2132     9180     2171   
2022-09-15         9952     5968  ...    7393   2132     9180     2171   
2022-09-16         9952     5968  ...    7393   2132     9180     2171   

Province_State                                                     
Admin2         Sweetwater  Teton Uinta Unassigned Washakie Weston  
2022-09-14          12117  11583  6102          0     2643   1812  
2022-09-15          12117  11583  6102          0     2643   1812  
2022-09-16          12117  11583  6102          0     2643   1812  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-09-14,20239,1364,34,31244,11970,95415,13230,11317,3080,0,...,27547,90577,4989,715,123,21646,14148,7351,15160,1888
2022-09-15,20322,1364,34,31244,11977,95537,13236,11329,3080,0,...,27553,90586,5001,715,123,21662,14148,7357,15185,1888
2022-09-16,20322,1364,34,31244,11983,95550,13240,11330,3086,0,...,27585,90639,5001,715,123,21679,14199,7364,15190,1888


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-09-14         226     702     101  107    254      54    129     647   
2022-09-15         227     705     101  107    256      54    129     650   
2022-09-16         227     705     101  107    256      54    129     650   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-09-14          167       87  ...     149     43       82       28   
2022-09-15          167       87  ...     149     43       82       28   
2022-09-16          167       87  ...     149     43       82       28   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-09-14            130    16    41          0       47     21  
2022-09-15            130    16    41          0       47     21  
2022-09-16            130    16    41          0       47     21  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change().round(7)
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-09-14,0.001767,0.000314,0.000070,0.000737,0.0,0.0,0.003789,0.0,0.0,0.000000,...,0.0,0.000049,0.000755,0.000180,0.000271,0.000000,0.0,0.0,0.000117,0.000136
2022-09-15,0.000605,0.000269,0.000052,0.000000,0.0,0.0,0.000000,0.0,0.0,0.002742,...,0.0,0.000053,0.000000,0.000234,0.000259,0.000252,0.0,0.0,0.000072,0.000000
2022-09-16,0.000620,0.000226,0.000055,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000658,...,0.0,0.000070,0.000587,0.000061,0.000269,0.000000,0.0,0.0,0.000078,0.000000


In [60]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [61]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change().round(7)
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-09-14,0.000128,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,NaN,0.0,0.0,0.0
2022-09-15,0.000000,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.011482,...,0.0,0.0,0.0,0.0,0.000116,0.000175,NaN,0.0,0.0,0.0
2022-09-16,0.000257,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.008548,...,0.0,0.0,0.0,0.0,0.000000,0.000000,NaN,0.0,0.0,0.0


In [62]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [63]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change().round(7)
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-09-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [65]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change().round(7)
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-09-14,0.000000,0.00409,0.0,0.00271,0.001081,0.000159,0.000531,0.000933,0.000574,0.0,...,0.000791,0.001164,0.00000,0.003851,0.001131,0.000815,0.000000,0.001290,0.000814,0.0
2022-09-15,0.005288,0.00000,0.0,0.00000,0.000895,0.004811,0.001423,0.000656,0.000763,0.0,...,0.000656,0.000135,0.00241,0.000000,0.000565,0.000811,0.000000,0.000924,0.000878,0.0
2022-09-16,0.000000,0.00000,0.0,0.00000,0.000699,0.000501,0.000381,0.001301,0.000835,0.0,...,0.000682,0.001303,0.00000,0.000000,0.000782,0.000886,0.002659,0.001159,0.000720,0.0


In [66]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [67]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change().round(7)
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2022-09-14      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2022-09-15      0.003839  0.004071  0.006885  0.004161  0.007197  0.001797   
2022-09-16      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

Province_State                                         ... Wyoming         \
Admin2            Butler   Calhoun Chambers  Cherokee  ...    Park Platte   
2022-09-14      0.000000  0.000000  0.00000  0.000000  ...     0.0    0.0   
2022-09-15      0.007699  0.012555  0.00272  0.006238  ...     0.0    0.0   
2022-09-16      0.000000  0.000000  0.00000  0.000000  ...     0.0    0.0   

Province_State                                                               \
Admin2         Sheridan Sublette Sweetwater Teton Uinta Unassigned Washakie   
2022-09-14          0.0      0.0        0.0   0.0   0.0        NaN      0.0   
2022-09-15          0.0      0.0        0.0   0.0   0.0        NaN      0.0   
2022-09-16          0.0      0.0        0.0   0.0   0.0        NaN      0.0   

Province_State         
Admin2         Weston  
2022-09-14        0.0  
2022-09-15        0.0  
2022-09-16        0.0  

[3 rows x 3336 columns]

In [68]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [69]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change().round(7)
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-09-14,0.000000,0.019432,0.0,0.002631,0.000501,0.000524,0.000756,0.000088,0.000000,NaN,...,0.000327,0.000530,0.000000,0.0,0.0,0.000693,0.000000,0.000953,0.000198,0.0
2022-09-15,0.004101,0.000000,0.0,0.000000,0.000585,0.001279,0.000454,0.001060,0.000000,NaN,...,0.000218,0.000099,0.002405,0.0,0.0,0.000739,0.000000,0.000816,0.001649,0.0
2022-09-16,0.000000,0.000000,0.0,0.000000,0.000501,0.000136,0.000302,0.000088,0.001948,NaN,...,0.001161,0.000585,0.000000,0.0,0.0,0.000785,0.003605,0.000951,0.000329,0.0


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change().round(7)
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                  \
Admin2           Autauga   Baldwin Barbour Bibb    Blount Bullock Butler   
2022-09-14      0.000000  0.000000     0.0  0.0  0.000000     0.0    0.0   
2022-09-15      0.004425  0.004274     0.0  0.0  0.007874     0.0    0.0   
2022-09-16      0.000000  0.000000     0.0  0.0  0.000000     0.0    0.0   

Province_State                              ... Wyoming                  \
Admin2           Calhoun Chambers Cherokee  ...    Park Platte Sheridan   
2022-09-14      0.000000      0.0      0.0  ...     0.0    0.0      0.0   
2022-09-15      0.004637      0.0      0.0  ...     0.0    0.0      0.0   
2022-09-16      0.000000      0.0      0.0  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-09-14          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2022-09-15          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2022-09-16          0.0        0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [72]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [73]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean().round(7)
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-09-14,0.001488,0.000309,0.000045,0.000376,0.000075,0.0,0.001894,0.000052,0.000742,0.000119,...,0.000214,0.000076,0.000532,0.000134,0.000255,8.000000e-07,0.0,9.000000e-07,0.000096,0.000096
2022-09-15,0.001046,0.000289,0.000048,0.000188,0.000038,0.0,0.000947,0.000026,0.000371,0.001431,...,0.000107,0.000065,0.000266,0.000184,0.000257,1.264000e-04,0.0,4.000000e-07,0.000084,0.000048
2022-09-16,0.000833,0.000258,0.000052,0.000094,0.000019,0.0,0.000474,0.000013,0.000186,0.001045,...,0.000053,0.000067,0.000427,0.000122,0.000263,6.320000e-05,0.0,2.000000e-07,0.000081,0.000024


In [74]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [75]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-09-14,0.000167,0.000106,1.000000e-07,0.000025,0.0,NaN,0.0,0.000030,0.000102,0.000417,...,0.000119,0.0,0.0,0.00004,0.000019,7.000000e-07,NaN,0.0,1.000000e-07,0.0
2022-09-15,0.000083,0.000053,0.000000e+00,0.000013,0.0,NaN,0.0,0.000015,0.000051,0.005949,...,0.000059,0.0,0.0,0.00002,0.000068,8.800000e-05,NaN,0.0,1.000000e-07,0.0
2022-09-16,0.000170,0.000026,0.000000e+00,0.000006,0.0,NaN,0.0,0.000007,0.000025,0.007248,...,0.000030,0.0,0.0,0.00001,0.000034,4.400000e-05,NaN,0.0,0.000000e+00,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [77]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean().round(7)
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-09-14,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-09-15,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-09-16,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078


In [78]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [79]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-09-14,0.000052,0.002060,0.000023,0.001361,0.000907,0.000181,0.000495,0.001453,0.000664,0.0,...,0.000434,0.000886,0.000018,0.001938,0.001082,0.000957,0.000202,0.001404,0.000805,0.000656
2022-09-15,0.002670,0.001030,0.000012,0.000681,0.000901,0.002496,0.000959,0.001055,0.000714,0.0,...,0.000545,0.000510,0.001214,0.000969,0.000823,0.000884,0.000101,0.001164,0.000842,0.000328
2022-09-16,0.001335,0.000515,0.000006,0.000340,0.000800,0.001499,0.000670,0.001178,0.000774,0.0,...,0.000614,0.000906,0.000607,0.000484,0.000803,0.000885,0.001380,0.001162,0.000781,0.000164


In [80]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [81]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2022-09-14      0.000047  0.000051  0.000081  0.000060  0.000036  0.000095   
2022-09-15      0.001943  0.002061  0.003483  0.002111  0.003617  0.000946   
2022-09-16      0.000972  0.001031  0.001741  0.001055  0.001808  0.000473   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2022-09-14      0.000073  0.000045  0.000044  0.000058  ...  0.000783   
2022-09-15      0.003886  0.006300  0.001382  0.003148  ...  0.000392   
2022-09-16      0.001943  0.003150  0.000691  0.001574  ...  0.000196   

Province_State                                                              \
Admin2            Platte Sheridan  Sublette Sweetwater     Teton     Uinta   
2022-09-14      0.000121  0.00044  0.000583   0.000398  0.000740  0.001485   
2022-09-15      0.000061  0.00022  0.000291   0.000199  0.000370  0.000743   
2022-09-16      0.000030  0.00011  0.000146   0.000100  0.000185  0.000371   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2022-09-14      -0.503937  0.000291  0.000710  
2022-09-15      -0.503937  0.000145  0.000355  
2022-09-16      -0.503937  0.000073  0.000178  

[3 rows x 3336 columns]

In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [83]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-09-14,0.000031,0.009716,0.0,0.001322,0.000345,0.000383,0.000608,0.000436,0.000042,NaN,...,0.000167,0.000332,0.000013,0.000044,0.001025,0.000763,0.000262,0.001069,0.000573,0.000534
2022-09-15,0.002066,0.004858,0.0,0.000661,0.000465,0.000831,0.000531,0.000748,0.000021,NaN,...,0.000192,0.000216,0.001209,0.000022,0.000512,0.000751,0.000131,0.000943,0.001111,0.000267
2022-09-16,0.001033,0.002429,0.0,0.000330,0.000483,0.000483,0.000416,0.000418,0.000984,NaN,...,0.000677,0.000400,0.000605,0.000011,0.000256,0.000768,0.001868,0.000947,0.000720,0.000133


In [84]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [85]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                       \
Admin2           Autauga   Baldwin Barbour      Bibb    Blount Bullock Butler   
2022-09-14      0.000105  0.000034     0.0  0.000149  0.000031     0.0    0.0   
2022-09-15      0.002265  0.002154     0.0  0.000074  0.003953     0.0    0.0   
2022-09-16      0.001133  0.001077     0.0  0.000037  0.001976     0.0    0.0   

Province_State                                   ... Wyoming                \
Admin2           Calhoun      Chambers Cherokee  ...    Park        Platte   
2022-09-14      0.000037  7.000000e-07      0.0  ...     0.0  4.000000e-07   
2022-09-15      0.002337  4.000000e-07      0.0  ...     0.0  2.000000e-07   
2022-09-16      0.001168  2.000000e-07      0.0  ...     0.0  1.000000e-07   

Province_State                                                               \
Admin2          Sheridan  Sublette    Sweetwater Teton     Uinta Unassigned   
2022-09-14      0.000024  0.000072  1.000000e-07   0.0  0.006250       -1.0   
2022-09-15      0.000012  0.000036  1.000000e-07   0.0  0.003125       -1.0   
2022-09-16      0.000006  0.000018  0.000000e+00   0.0  0.001563       -1.0   

Province_State                     
Admin2         Washakie    Weston  
2022-09-14          0.0  0.000098  
2022-09-15          0.0  0.000049  
2022-09-16          0.0  0.000024  

[3 rows x 3336 columns]

In [86]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [87]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

HTTPError: HTTP Error 403: Forbidden

In [ ]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [ ]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

In [ ]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [ ]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

In [ ]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

#### 